In [ ]:
#Libraries
from bs4 import BeautifulSoup
import requests
import numpy as np 
import time
import re
import tweepy as tp

#credentials to login to twitter api
consumer_key = '6DI6Z1dgRHDvrWXQCfgRKBaWn'
consumer_secret = 'HYdadkisuka2XQdttE186rK3ZXwZwIczrWvfGan79kiud2JI1a'
access_token = '954192403753086976-9BXH66iOhTbvlzeANEDwoMyHTF5RQLx'
access_secret = 'ijBGtHnu3dKr1jzzeKjOgs4prTfQtb3GC498rnrSqwXfc'

def flavor_tweet():   
    while True:
        try:
            print ('............ Looking for flavors ............... \n ............................................... \n')
            #Pull Flavors
            menu_url  = requests.get("http://linkedin.cafebonappetit.com/cafe/urban-terrace/")
            menu_data = menu_url.text
            menu_soup = BeautifulSoup(menu_data, 'html.parser') 
            menu_soup_daypart = menu_soup.select_one('body > div > #page > #main > #primary > #content > #panel-daypart-menu-1')
            menu_soup_script = menu_soup_daypart.select_one('script')
            menu_items = menu_soup_script.text.split('"monotony":{}}')
            menu_items_icecream = list(filter(lambda x: re.findall('"station_id":"10167"',x), menu_items))
            flavors = list(map(lambda x: re.findall('"label":"(.*?)".*"station_id":"10167"',x), menu_items_icecream))
        except Exception as e:
            flavors = []
            pass  
        #Remove hard coded flavors
        remove_list = [('butter pecan ice cream', 'butter pecan ice cream'), ('chocolate ice cream', 'chocolate ice cream'), ('vanilla ice cream', 'vanilla ice cream'), ('caramel sauce', 'caramel sauce'), ('vegan chocolate syrup', 'vegan chocolate syrup'), ('whipped cream', 'whipped cream')]

        #Flavors
        main_list = np.setdiff1d(flavors,remove_list) #Flavors are here ['flavor1','flavor2]

        if len(main_list) > 0:
            #Login to twitter account api
            auth = tp.OAuthHandler( consumer_key, consumer_secret )
            auth.set_access_token( access_token, access_secret )
            api = tp.API(auth)
            #Tweet
            yummy = 'New flavors are out! \n* ' + ' \n* '.join(main_list)
            api.update_status(yummy)
            print(yummy + '\n')
            print('............................................... \n We Just Tweeted! Lets check back tomorrow ... \n ............................................... \n')
            time.sleep(1200 * 60) #20 hours
        else:
            print('...............No new Flavors .................. \n ............................................... \n ...............Checking in 20min ..................')
            time.sleep(20 * 60) #20minutes
            
if __name__ == '__main__':
    flavor_tweet()
    

In [ ]:
import csv
def update_flavor():   
    #Read the latest flavors from csv
    with open('iceCreamFlavors.csv', 'rb') as f:
        reader = csv.reader(f)
        updated_flavors = list(reader)  
        latest_flavor = updated_flavors[-1]

    #Create new Flavors in JSON from csv
    new_flavors = [] #This will be used for flask ('flavor1',flavor1'),('flavor2','flavor2')
    for i in latest_flavor:
        new_flavors.append((str(i),str(i)))           

    #Update flavors and replace
    yummy = 'Current flavors! \n* ' + ' \n* '.join(latest_flavor)
    print(yummy + '\n')
    existing_flavors = new_flavors
    return existing_flavors

update_flavor()